## Coursera Capstone assignment - Toronto neighbourhood Clustering & Segmenting - Part 3 (Final)

This notebook is created as part of Corusera Capstone project. This notebook is created as part of capstone project  assignment - 'Clustering & Segmenting Toronto city data'. This notebook implements the following:

1. Extract the post codes & Neighbourhood data correspoding to various bouroughs in Toronto, ontario from a wikipedia page using python library - BeautifulSoup
    
2. Store the extracted data into a dataframe with columns as below:
    - Postal Code
    - Borough
    - Neighbourhood
    
3. Update the above created dataframe with the Latitude & Longitudes details

4. **Cluster and Segment the Toronto neighbourhoods data similar to the Newyork neighbourhoods data excercise.**

**Note:**
The overall Toronto dataset will be sliced to work with only "Toronto" based bouroughs (ie boroughs with word "Toronto" in them.)



### Import libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#import BeautifulSoup library for parsing wikipedia page
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#import geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


print('Libraries imported.')

Libraries imported.


### This section repeats the neighbourhood data dataset creation from assignment part 1 & part 2 ie scraping the wikipage & storing it in dataframe & adding latitude & longitude details 

In [3]:
wiki_page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_doc =  requests.get(wiki_page)
wikipage = BeautifulSoup(html_doc.text, 'html.parser') #read the extracted Wikipage using beautifulsoup
table = wikipage.find("table")
i=0
j=0
temp_dict=dict()
for row in table.findAll("tr"): 
    col = row.findAll("td")
    if len(col)>0:
        pcode = col[0].get_text().strip('\n')
        borough = col[1].get_text().strip('\n')
        neighbourhood = col[2].get_text().strip('\n')
        dict_key=pcode+'_'+borough
        
        if neighbourhood == 'Not assigned':
            neighbourhood = borough # to handle neighbouhoods that are 'Not assgined' & assign them to borough value in such cases
            
        if borough != 'Not assigned':  #skips all boroughs which are having value = 'Not assigned'
            #group the neighbourhoods belonging to same postal area in a comma separated csv
            if dict_key in temp_dict:
                temp_dict[dict_key]=temp_dict[dict_key]+','+neighbourhood
            else:
                temp_dict[dict_key]=neighbourhood
        else:
            j=j+1
        i=i+1
#priting howmany entries skipped        
text = 'Skipped {} boroughs out of {} entries as they are <Not assigned>\n'.format(j,i)
print(text)

#convert dictionary to a temporary dataframe
temp_df = pd.DataFrame(temp_dict.items(),columns=['key','Neighbourhood'])
#Split the key column to separate columns
temp_df[['Postal Code','Borough']] = temp_df['key'].str.split('_', n=1, expand=True)
# drop the key column
temp_df.drop('key',axis=1,inplace=True)

#Rearrange the columns and store the data to the base dataframe for toronto neighbourhoods
toronto_nhood_data = temp_df[['Postal Code','Borough','Neighbourhood']]

# fetch the geospatial data from the provide web location into a pandas dataframe
lat_long_data=pd.read_csv('http://cocl.us/Geospatial_data')
print("Lat-long data fetched\n")
print("Sorting the lat-long dataframe with PostCode...")
lat_long_data = lat_long_data.sort_values(by=['Postal Code'])
print("Dropping the Postal Code column...")
lat_long_data.drop('Postal Code',axis=1,inplace=True)

#Sort the toronto neighbourhood data with postal code
toronto_nhood_data=toronto_nhood_data.sort_values(by=['Postal Code'])
toronto_nhood_data.reset_index(drop=True,inplace=True)

#Add the latlong dataframe into toronto neighbourhood dataframe to get the final dataframe
toronto_nhood_data [['Latitude','Longitude']] = lat_long_data

print("Toronto neighbourhoods dataframe created!!\n")
print('Dataframe shape:',toronto_nhood_data.shape,'\n')
toronto_nhood_data.head()

Skipped 77 boroughs out of 289 entries as they are <Not assigned>

Lat-long data fetched

Sorting the lat-long dataframe with PostCode...
Dropping the Postal Code column...
Toronto neighbourhoods dataframe created!!

Dataframe shape: (103, 5) 



,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Fetch the latitude longitude for Toronto using geopy

In [6]:
address = 'Toronto,Ontario'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto,Ontario = {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto,Ontario = 43.653963, -79.387207.


###  Create a map of Toronto with all neighbourhoods overlaid

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood,pcode in zip(toronto_nhood_data['Latitude'], toronto_nhood_data['Longitude'], toronto_nhood_data['Borough'], toronto_nhood_data['Neighbourhood'],toronto_nhood_data['Postal Code'],):
    label = '{}, {},{}'.format(pcode,neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Slice the original Toronto neighborhood dataframe to a new dataframe to work with only Toronto Boroughs 

In [13]:
toronto_slice_data = toronto_nhood_data[toronto_nhood_data['Borough'].str.contains("Toronto")].reset_index(drop=True)
print ("New dataframe shape:",toronto_slice_data.shape)
toronto_slice_data.head()

New dataframe shape: (38, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Display only the sliced data in the map

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto_new = folium.Map(location=[latitude, longitude], zoom_start=11.5)

# add markers to map
for lat, lng, borough, neighborhood,pcode in zip(toronto_slice_data['Latitude'], toronto_slice_data['Longitude'], toronto_slice_data['Borough'], toronto_slice_data['Neighbourhood'],toronto_slice_data['Postal Code'],):
    label = '{}, {},{}'.format(pcode,neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto_new)  
    
map_toronto_new

### Explore the neighbourhoods using Foursquare API

As part of this , we will try to fetch 100 venues within 500 mtrs of radius from each neibhourhood record in the dataframe using foursquare API

#### Define foursquare creds (intentionally removed)

In [25]:
#removed

#### Define a function to fetch a maximum of 100 venues within 500mtrs radius 

In [35]:
def getNearbyVenues(names, neighborhoods,latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name,nhood, lat, lng in zip(names, neighborhoods,latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            nhood,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood Postal Code',
                  'Neighborhoods',          
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Call the function to fetch venues from Toronto & store in a new dataframe

In [37]:
toronto_venues = getNearbyVenues(names=toronto_slice_data['Postal Code'],
                                    neighborhoods= toronto_slice_data['Neighbourhood'],
                                   latitudes=toronto_slice_data['Latitude'],
                                   longitudes=toronto_slice_data['Longitude']
                                  )
toronto_venues.head()

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y


,Neighborhood Postal Code,Neighborhoods,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,M4E,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,M4E,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,M4E,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [39]:
print(toronto_venues.shape)

(1692, 8)


In [43]:
# Groupby neighbourhoods
toronto_venues.groupby('Neighborhood Postal Code').count()

,Neighborhoods,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood Postal Code,,,,,,,
M4E,4,4,4,4,4,4,4
M4K,43,43,43,43,43,43,43
M4L,20,20,20,20,20,20,20
M4M,40,40,40,40,40,40,40
M4N,4,4,4,4,4,4,4
M4P,8,8,8,8,8,8,8
M4R,21,21,21,21,21,21,21
M4S,34,34,34,34,34,34,34
M4T,1,1,1,1,1,1,1


#### Find out how many unique categories can be curated from all the returned venues

In [44]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


### Analyse the neighbourhoods

In [66]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood postal code column back to dataframe
toronto_onehot['Neighborhood Postal Code'] = toronto_venues['Neighborhood Postal Code'] 

# move neighborhood postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print("Dataframe shape: ",toronto_onehot.shape)
toronto_onehot.head()

Dataframe shape:  (1692, 235)


,Neighborhood Postal Code,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [67]:
toronto_grouped = toronto_onehot.groupby('Neighborhood Postal Code').mean().reset_index()
print("Dataframe shape: ",toronto_grouped.shape)
toronto_grouped.head()

Dataframe shape:  (38, 235)


,Neighborhood Postal Code,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.250000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.00

#### Print each neighborhood along with the top 5 most common venues

In [68]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood Postal Code']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0  Health Food Store  0.25
1        Coffee Shop  0.25
2                Pub  0.25
3       Neighborhood  0.25
4     Adult Boutique  0.00


----M4K----
                venue  freq
0    Greek Restaurant  0.23
1         Coffee Shop  0.09
2      Ice Cream Shop  0.07
3  Italian Restaurant  0.05
4           Bookstore  0.05


----M4L----
               venue  freq
0               Park  0.10
1          Pet Store  0.05
2        Coffee Shop  0.05
3       Burger Joint  0.05
4  Food & Drink Shop  0.05


----M4M----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2   Italian Restaurant  0.05
3            Gastropub  0.05
4  American Restaurant  0.05


----M4N----
                venue  freq
0                Park  0.25
1         Swim School  0.25
2  Dim Sum Restaurant  0.25
3            Bus Line  0.25
4      Adult Boutique  0.00


----M4P----
               venue  freq
0     Sandwich Place  0.12
1     Clothing Store  0.12
2  

#### Creating new dataframe with the above data

In [69]:
# function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood Postal Code' ] = toronto_grouped['Neighborhood Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Health Food Store,Coffee Shop,Pub,Neighborhood,Cosmetics Shop,Convenience Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,M4L,Park,Pet Store,Sushi Restaurant,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Movie Theater
3,M4M,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Gastropub,Chinese Restaurant,Fish Market,Juice Bar,Bookstore
4,M4N,Park,Swim School,Bus Line,Dim Sum Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


### Clustering Neighborhoods

#### Run kmeans with 5 clusters

In [115]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0], dtype=int32)

#### create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [116]:
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_slice_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood Postal Code'), on='Postal Code')

toronto_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Neighborhood,Cosmetics Shop,Convenience Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Park,Pet Store,Sushi Restaurant,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Gastropub,Chinese Restaurant,Fish Market,Juice Bar,Bookstore
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Dim Sum Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Visualize the clusters

In [117]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, pcode,nhood, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Neighbourhood'],toronto_merged['Cluster Labels']):
    poi = pcode+','+nhood
    label = folium.Popup(str(poi) + ': Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Display Clusters

#### Cluster 1

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] +[1]+[2]+ list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,0,Health Food Store,Coffee Shop,Pub,Neighborhood,Cosmetics Shop,Convenience Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",0,Park,Pet Store,Sushi Restaurant,Brewery,Burger Joint,Burrito Place,Sandwich Place,Pub,Coffee Shop,Movie Theater
3,M4M,East Toronto,Studio District,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Gastropub,Chinese Restaurant,Fish Market,Juice Bar,Bookstore
4,M4N,Central Toronto,Lawrence Park,0,Park,Swim School,Bus Line,Dim Sum Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,M4P,Central Toronto,Davisville North,0,Burger Joint,Park,Gym,Sandwich Place,Breakfast Spot,Hotel,Clothing Store,Food & Drink Shop,Yoga Studio,Doner Restaurant
6,M4R,Central Toronto,North Toronto West,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Sandwich Place,Rental Car Location,Dessert Shop,Mexican Restaurant,Furniture / Home Store,Metro Station
7,M4S,Central Toronto,Davisville,0,Sandwich Place,Dessert Shop,Pizza Place,Café,Coffee Shop,Italian Restaurant,Pharmacy,Sushi Restaurant,Gym,Brewery
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0,Coffee Shop,Pub,American Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,Supermarket,Sushi Restaurant,Pizza Place,Vietnamese Restaurant
11,M4X,Downtown Toronto,"Cabbagetown,St. James Town",0,Coffee Shop,Restaurant,Café,Italian Restaurant,Bakery,Pub,Pizza Place,Park,Pharmacy,Breakfast Spot


#### Cluster 2

In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] +[1]+[2]+ list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M5N,Central Toronto,Roselawn,1,Home Service,Garden,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 3

In [120]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] +[1]+[2]+ list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4T,Central Toronto,"Moore Park,Summerhill East",2,Playground,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 4

In [121]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] +[1]+[2]+ list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M4W,Downtown Toronto,Rosedale,3,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 5

In [122]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] +[1]+[2]+ list(range(5, toronto_merged.shape[1]))]]

,Postal Code,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,M5P,Central Toronto,"Forest Hill North,Forest Hill West",4,Trail,Sushi Restaurant,Jewelry Store,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
